# Feature 9: pases en profundidad completos p90

In [1]:
# librearías
import pandas as pd
import numpy as np

In [2]:
pases = pd.read_csv(r'C:\Users\DELUX\Desktop\consultoria_futbol\prototipo\versiones3\pases.csv',low_memory=False)

In [3]:
pases = pases.drop(pases[(pases['outcome']==0)| #descartamos los pases incompletos
                        (pases['balón en profundidad']==False)].index)
pases = pases[['fecha','equipo','id_jugador','jugador','coord_x_0','coord_x_1','balón en profundidad']] #descartamos las columnas que no nos interesan

In [4]:
pases.columns

Index(['fecha', 'equipo', 'id_jugador', 'jugador', 'coord_x_0', 'coord_x_1',
       'balón en profundidad'],
      dtype='object')

In [5]:
pases.head()

,fecha,equipo,id_jugador,jugador,coord_x_0,coord_x_1,balón en profundidad
237,1,Girona,61595.0,Marc Muniesa,3,5,True
956,1,Huesca,81045.0,Samuele Longo,4,5,True
1806,1,Barcelona,19054.0,Lionel Messi,4,6,True
2146,1,Espanyol,169007.0,Sergi Darder,4,5,True
2232,1,Celta de Vigo,40270.0,Iago Aspas,5,6,True


In [6]:
mtr9 = pases.groupby(['fecha','equipo','id_jugador','jugador']).count().reset_index().drop(['coord_x_0','coord_x_1'],1).rename(columns={'balón en profundidad':'mtr9'})

In [7]:
mtr9.head()

,fecha,equipo,id_jugador,jugador,mtr9
0,1,Athletic Club,76012.0,Iker Muniain,1
1,1,Atlético de Madrid,76650.0,Antoine Griezmann,1
2,1,Barcelona,19054.0,Lionel Messi,3
3,1,Celta de Vigo,40270.0,Iago Aspas,1
4,1,Espanyol,169007.0,Sergi Darder,1


In [8]:
mtr9[mtr9['jugador']=='Lionel Messi'].head()

,fecha,equipo,id_jugador,jugador,mtr9
2,1,Barcelona,19054.0,Lionel Messi,3
11,2,Barcelona,19054.0,Lionel Messi,2
21,3,Barcelona,19054.0,Lionel Messi,3
27,4,Barcelona,19054.0,Lionel Messi,1
39,5,Barcelona,19054.0,Lionel Messi,1


In [9]:
mtr9 = mtr9.groupby(['equipo','id_jugador','jugador']).sum().reset_index().drop('fecha',1)

In [10]:
mtr9.head()

,equipo,id_jugador,jugador,mtr9
0,Alavés,92190.0,Jonathan Rodríguez Menéndez,2
1,Alavés,103263.0,Takashi Inui,2
2,Alavés,168287.0,Jonathan Calleri,1
3,Athletic Club,18498.0,Raúl García,3
4,Athletic Club,38394.0,Beñat Etxebarria,1


In [11]:
mtr9[mtr9['jugador']=='Lionel Messi']

,equipo,id_jugador,jugador,mtr9
23,Barcelona,19054.0,Lionel Messi,31


In [12]:
mtr9.sort_values('mtr9',ascending=False).head(10)

,equipo,id_jugador,jugador,mtr9
23,Barcelona,19054.0,Lionel Messi,31
67,Girona,63904.0,Borja García,10
56,Espanyol,234370.0,Marc Roca,10
60,Getafe,76555.0,Jorge Molina,8
88,Levante,86168.0,Rubén Rochina,8
153,Sevilla,100731.0,Franco Vázquez,8
19,Atlético de Madrid,156223.0,Ángel Correa,7
61,Getafe,80795.0,Francisco Portillo,6
111,Real Betis,200826.0,Giovani Lo Celso,6
6,Athletic Club,76012.0,Iker Muniain,6


In [13]:
dataAggJug = pd.read_csv(r'C:\Users\DELUX\Desktop\consultoria_futbol\comparativoJugadores\dataAggJug.csv')

In [14]:
dataAggJug.head()

,fecha,id_jugador,estadistica,num_estadistica,jugador
0,1,212769,diving_save,1,Unai Simón
1,1,212769,leftside_pass,2,Unai Simón
2,1,212769,accurate_pass,10,Unai Simón
3,1,212769,rightside_pass,1,Unai Simón
4,1,212769,attempts_conceded_ibox,9,Unai Simón


In [15]:
minJug = dataAggJug[dataAggJug['estadistica']=='mins_played']

In [16]:
minJug = minJug.drop('estadistica',1).rename(columns={'num_estadistica':'minJug'})

In [17]:
minJug.head()

,fecha,id_jugador,minJug,jugador
34,1,212769,90,Unai Simón
111,1,59062,65,Óscar De Marcos
180,1,197319,90,Yeray Álvarez
249,1,242831,90,Peru Nolaskoain
328,1,42670,90,Yuri Berchiche Izeta


In [18]:
minJug[minJug['jugador']=='Ricard Puig']

,fecha,id_jugador,minJug,jugador
536458,32,437765,67,Ricard Puig
607246,36,437765,90,Ricard Puig


In [19]:
minJug = minJug.groupby(['id_jugador','jugador']).sum().reset_index().drop('fecha',1)

In [20]:
minJug.head()

,id_jugador,jugador,minJug
0,7261,Daniele Bonera,233
1,10316,Joaquín Sánchez Rodríguez,1694
2,11129,David López,1820
3,11602,Borja Fernández,839
4,14028,Juan José Camacho Barnola,34


In [21]:
minJug[minJug['jugador']=='Iván López Álvarez']

,id_jugador,jugador,minJug
382,181859,Iván López Álvarez,17


In [22]:
mtr9x90 = minJug.merge(mtr9,on=['id_jugador','jugador'],how='left') #Hacemos left join para tener tabla con minutos jugados por jugador por fecha
mtr9x90['mtr9x90'] = mtr9x90['mtr9']/mtr9x90['minJug']*90 #Creamos mtr9x90
mtr9x90 = mtr9x90.drop(['mtr9','minJug'],1)

In [23]:
mtr9x90.head(10)

,id_jugador,jugador,equipo,mtr9x90
0,7261,Daniele Bonera,NaN,NaN
1,10316,Joaquín Sánchez Rodríguez,NaN,NaN
2,11129,David López,NaN,NaN
3,11602,Borja Fernández,NaN,NaN
4,14028,Juan José Camacho Barnola,NaN,NaN
5,14464,Aritz Aduriz,NaN,NaN
6,14725,Gorka Iraizoz,NaN,NaN
7,15943,Thomas Vermaelen,NaN,NaN
8,16021,Diego López,NaN,NaN
9,16261,Iván Cuéllar,NaN,NaN


In [24]:
mtr9x90.isnull().sum()

id_jugador      0
jugador         0
equipo        353
mtr9x90       353
dtype: int64

In [25]:
mtr9x90['mtr9x90'] = mtr9x90['mtr9x90'].replace(np.nan, 0)

In [26]:
mtr9x90.head()

,id_jugador,jugador,equipo,mtr9x90
0,7261,Daniele Bonera,NaN,0.0
1,10316,Joaquín Sánchez Rodríguez,NaN,0.0
2,11129,David López,NaN,0.0
3,11602,Borja Fernández,NaN,0.0
4,14028,Juan José Camacho Barnola,NaN,0.0


In [27]:
mtr9x90.isnull().sum()

id_jugador      0
jugador         0
equipo        353
mtr9x90         0
dtype: int64

In [28]:
mtr9x90[mtr9x90['jugador']=='Raúl García']

,id_jugador,jugador,equipo,mtr9x90
19,18498,Raúl García,Athletic Club,0.113493
161,80908,Raúl García,NaN,0.000000


In [29]:
feat9 = mtr9x90.copy()
feat9 = feat9.rename(columns={'mtr9x90':'feat9'})

In [30]:
feat9.sort_values('feat9',ascending=False).head(20)

,id_jugador,jugador,equipo,feat9
530,481655,Martín Zubimendi,Real Sociedad,15.000000
22,19054,Lionel Messi,Barcelona,1.029520
502,437765,Ricard Puig,Barcelona,0.573248
184,86168,Rubén Rochina,Levante,0.397351
126,63904,Borja García,Girona,0.341426
330,156223,Ángel Correa,Atlético de Madrid,0.328125
256,100731,Franco Vázquez,Sevilla,0.314823
467,227164,Manuel Morlanes,Villarreal,0.294599
471,234370,Marc Roca,Espanyol,0.288832
226,90801,Ignasi Miquel,Getafe,0.283912


In [31]:
feat9[feat9['equipo']=='Atlético de Madrid']

,id_jugador,jugador,equipo,feat9
36,32281,Diego Godín,Atlético de Madrid,0.035899
144,76650,Antoine Griezmann,Atlético de Madrid,0.112430
148,77390,Jorge Resurrección,Atlético de Madrid,0.139157
188,86399,Víctor Machín Pérez,Atlético de Madrid,0.241935
198,88482,Álvaro Morata,Atlético de Madrid,0.082645
211,89335,Saúl Ñíguez,Atlético de Madrid,0.061538
318,151883,José Giménez,Atlético de Madrid,0.050790
330,156223,Ángel Correa,Atlético de Madrid,0.328125
345,167199,Thomas Partey,Atlético de Madrid,0.130814
346,167449,Thomas Lemar,Atlético de Madrid,0.147783


In [32]:
feat9.isnull().sum()

id_jugador      0
jugador         0
equipo        353
feat9           0
dtype: int64

In [33]:
cd C:\Users\DELUX\Desktop\publicaciones\1_DataScoutingClustering\1_MedioCentroCreativo\df

C:\Users\DELUX\Desktop\publicaciones\1_DataScoutingClustering\1_MedioCentroCreativo\df


In [34]:
feat9.to_csv('feat9.csv',encoding='utf-8', index=False,decimal=',')